<h1><center>MOVIE RECOMMENDER SYSTEM AND SENTIMENT ANALYSIS ON
MOVIE REVIEWS
</center></h1>


### Sentiment Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import numpy as np

reviews_data = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

nltk.download('stopwords')

reviews_data["review_content"] = reviews_data["review_content"].fillna(" ")
reviews_data["review_content"].isna().sum()

stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])

reviews_data["review_score"].isna().sum()
reviews_data["review_type"].value_counts()

reviews_data = reviews_data[reviews_data['review_type'].notna()]
reviews_data['sentiment'] = reviews_data['review_type'].apply(lambda type : 1 if type == "Fresh" else -1)                            

reviews_data.head()

reviews_data = reviews_data.drop(["critic_name", "top_critic", "publisher_name", "review_score", "review_date", "review_type"],1)
reviews_data["review_content"] = reviews_data["review_content"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

reviews_data['sentiment'] = reviews_data['sentiment'].replace({-1 : 'negative'})
reviews_data['sentiment'] = reviews_data['sentiment'].replace({1 : 'positive'})
fig = px.histogram(reviews_data, x="sentiment")
fig.update_traces(marker_color="indianred",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Product Sentiment')
fig.show()

links = []

review_percentage = pd.DataFrame()
grouped = reviews_data.groupby(reviews_data["rotten_tomatoes_link"])
pos_per = []
for i in reviews_data["rotten_tomatoes_link"].unique():
        links.append(i)
        r = grouped.get_group(i)["sentiment"].value_counts()
        sum_ = r.sum()
        if(r.index[0] == "positive"):
            pos = r.values[0]
            if(r.shape[0] == 2):
                neg = r.values[1]
            else:
                neg = 0
        elif(r.index[0] == "negative"):
            neg = r.values[0]
            if(r.shape[0] == 2):
                pos = r.values[1]
            else:
                pos = 0
        pos_per.append(round(pos/sum_*100))

reviews_percentage = grouped.head(len(grouped))
reviews_percentage = reviews_percentage.drop_duplicates(subset=['rotten_tomatoes_link']).reset_index(drop=True)
reviews_percentage = reviews_percentage.drop(["sentiment"],1)
reviews_percentage["percentage pos"] = pos_per
reviews_percentage = reviews_percentage.sort_values(by="percentage pos", ascending=False)
reviews_percentage.head()


def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final
reviews_data['review_content'] = reviews_data['review_content'].apply(remove_punctuation)

reviewsNew = reviews_data[['review_content','sentiment']]
reviewsNew.head()


index = reviews_data.index
reviews_data['random_number'] = np.random.randn(len(index))
train = reviews_data[reviews_data['random_number'] <= 0.8]
test = reviews_data[reviews_data['random_number'] > 0.8]

# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['review_content'])
test_matrix = vectorizer.transform(test['review_content'])

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()


X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

lr.fit(X_train,y_train)

predictions = lr.predict(X_test)


from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(predictions,y_test))

### Prepare datasets for recommendation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

movies_info = pd.read_csv('rotten_tomatoes_movies.csv')
for column in movies_info.columns:
    movies_info[column] = movies_info[column].astype(str).str.lower()
movies_info = movies_info[['movie_title','genres','directors','actors','movie_info','rotten_tomatoes_link']]

movies_data = reviews_percentage.merge(movies_info, how='inner', on='rotten_tomatoes_link')

genres_df = pd.DataFrame()
for i in range(movies_data.shape[0]):
    res = re.findall(r'[^,]+', movies_data["genres"].loc[i])
    genre_movies = [movies_data["movie_title"].loc[i]] * len(res)
    rotten_tom_link = [movies_data["rotten_tomatoes_link"].loc[i]] * len(res)
    score_movies = [movies_data["percentage pos"].loc[i]] * len(res)
    genres_df_part = pd.DataFrame(list(zip(res, genre_movies, rotten_tom_link, score_movies)), columns = ["genres", "movie title", "rotten tomatoes link", "percentage pos"]) 
    genres_df = pd.concat([genres_df,genres_df_part])

actors_df = pd.DataFrame()
for i in range(movies_data.shape[0]):
    res = re.findall(r'[^,]+', movies_data["actors"].loc[i])
    actor_movies = [movies_data["movie_title"].loc[i]] * len(res)
    rotten_tom_link = [movies_data["rotten_tomatoes_link"].loc[i]] * len(res)
    score_movies = [movies_data["percentage pos"].loc[i]] * len(res)
    actors_df_part = pd.DataFrame(list(zip(res, actor_movies, rotten_tom_link, score_movies)), columns = ["actors", "movies that participated", "rotten tomatoes link", "percentage pos"]) 
    actors_df = pd.concat([actors_df,actors_df_part])
    actors_df.head()

directors_df = pd.DataFrame()
for i in range(movies_data.shape[0]):
    res = re.findall(r'[^,]+', movies_data["directors"].loc[i])
    director_movies = [movies_data["movie_title"].loc[i]] * len(res)
    rotten_tom_link = [movies_data["rotten_tomatoes_link"].loc[i]] * len(res)
    score_movies = [movies_data["percentage pos"].loc[i]] * len(res)
    directors_df_part = pd.DataFrame(list(zip(res, director_movies, rotten_tom_link, score_movies)), columns = ["directors", "movies that participated", "rotten tomatoes link", "percentage pos"]) 
    directors_df = pd.concat([directors_df,directors_df_part])
    directors_df.head()


movies_data.directors = movies_data.directors.str.replace(' ','')
movies_data.actors = movies_data.actors.str.replace(' ','')
movies_data.head()

movies_data['recommend_solver_movie'] = movies_data['genres']+' '+movies_data['directors']+' '+movies_data['actors']+' '+movies_data['movie_info']

movies_data.drop(['genres','directors','actors','movie_info'], axis=1,inplace=True)

movies_data.set_index('movie_title',inplace=True)

movies_data.head()

movies_data["recommend_solver_movie"].isna().sum()

count_movie = CountVectorizer()
count_matrix_movie = count_movie.fit_transform(movies_data['recommend_solver_movie'])
cosine_sim_movie = cosine_similarity(count_matrix_movie,count_matrix_movie)

In [ ]:
our_stopword = ['the', 'and']

def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word.lower() not in our_stopword:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array


def movie_recommender(title, cosine_sim = cosine_sim_movie):
    recommended_movies = []
    movie_score = []
    indices = pd.Series(movies_data.index)
    idx = indices[indices == title].index[0]
    recommend = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    recommend = recommend[1:21]
    top_10_index = recommend.index
    for i in top_10_index:
        recommended_movies.append(list(movies_data.index)[i])
        movie_score.append(movies_data["percentage pos"][i])
    suggested_movies = list(zip(recommended_movies, movie_score))
    return suggested_movies
    
def title_recommender(title):
    recommended_movies = []
    movie_score = []
    count_title = CountVectorizer()
    movies_no_sw = remove_stopwords(movies_data.index)
    count_matrix_title = count_title.fit_transform(movies_no_sw)
    count_input = count_title.transform([title])
    cosine_sim_title = cosine_similarity(count_matrix_title, count_input)
    recommend = pd.Series(cosine_sim_title.flatten()).sort_values(ascending = False)
    recommend = recommend[0:20]
    top_10_index = recommend.index
    for i in top_10_index:
        recommended_movies.append(list(movies_data.index)[i])
        movie_score.append(movies_data["percentage pos"][i])
    suggested_movies = list(zip(recommended_movies, movie_score))
    return suggested_movies

### App (Tkinter)

In [ ]:
def openNewWindowMovie():
    
    def recommender_function(movie_input):
        disp_tf.delete('1.0', END)
        
        count_ = CountVectorizer()
        count_matrix = count_.fit_transform(movies_data.index)
        input_words = movie_input.split()
        
        search = 0
        for word in input_words:
            word = remove_punctuation(word)
            if word in count_.get_feature_names():
                search = 1
                break
                
        print(search)
        
        if(search == 1):
            if(movie_input in movies_data.index.unique()):
                recomm = movie_recommender(movie_input, cosine_sim_movie)
                for x,y in recomm:
                    disp_tf.insert(END, x.title() + " | Critic's Score: " + str(y) + "%" + '\n\n')
            else:
                if(movie_input != "the" and movie_input != "and"):
                    recomm = title_recommender(movie_input)
                    for x,y in recomm:
                        disp_tf.insert(END, x.title() + " | Critic's Score: " + str(y) + "%" + '\n\n')
                else:
                    disp_tf.insert(END, "Sorry, there are no results for your entry!")
        else:
            print(movie_input)
            disp_tf.insert(END, "Sorry, there are no results for your entry!")
            
    
    def clear_name():
        name_tf.delete(0, END)   
        
    def toggle_state(*_):
        if name_tf.var.get():
            btn['state'] = 'normal'
        else:
            btn['state'] = 'disabled'
        
    def recommend():
        if (name_tf.get() == ""):
            disp_tf.insert(END, "There is no entry!")
        else:
            input_ = name_tf.get().lower()
            recommender_function(input_)
            clear_name()

     
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    newWindowMovie = Toplevel(root)
    newWindowMovie.configure(bg='black')
    newWindowMovie.title("Movie Recommender")
    newWindowMovie.geometry("700x700")
   
    space = Label(newWindowMovie, text="", bg = "black").pack()
    space = Label(newWindowMovie, text="", bg = "black").pack()

    name_tf = tk.Entry(newWindowMovie)
    name_tf.var = tk.StringVar()
    name_tf['textvariable'] = name_tf.var
    name_tf.var.trace_add('write', toggle_state)
    
    name_lbl = Label(
    newWindowMovie,
    text='Enter Movie Title',
    bg='#132530',
    fg='white', 
    font = button_font)
    
    name_lbl.pack()
    space = Label(newWindowMovie, text="", bg = "black").pack()
    name_tf.pack()
    
    space = Label(newWindowMovie, text="", bg = "black").pack()

    btn = Button(
    newWindowMovie,
    text='Suggestions by title',
    relief=SOLID, state = 'disabled', activebackground='#bc272c',activeforeground='white', bg='#132530', fg='White',
    command=recommend, font=button_font)
    btn.pack(pady=10)
    
    space = Label(newWindowMovie, text="", bg = "black").pack()
    
    disp_tf = Text(newWindowMovie)
    disp_tf.config(font = button_font, bg = "#132530", fg = "white")
    disp_tf.pack(pady=5)

In [ ]:
def openNewWindowActor():
    
    def actors_recommender(actor_input):
        disp_tf.delete('1.0', END)
        
        if(actor_input in actors_df["actors"].values):
            actor_list = actors_df[actors_df["actors"] == actor_input].values.tolist()
            num = 0
            for x,y,z,t in actor_list:
                if(num <20): 
                    disp_tf.insert(END, y.title() + " | Critic's Score: " + str(t) + "%"  + '\n\n')
                    num = num +1
                else:
                    break
        else:
            disp_tf.insert(END, "Sorry, there are no results for your entry!")
                
    def toggle_state(*_):
        if actor_tf.var.get():
            btn['state'] = 'normal'
        else:
            btn['state'] = 'disabled'
                
    def clear_actor():
        actor_tf.delete(0, END)            
                
    def actor_show():
        if (actor_tf.get() == ""):
            disp_tf.insert(END, "There is no entry!")
        else:
            input_ = actor_tf.get().lower()
            actors_recommender(input_)
            clear_actor()
       
    newWindowActor = Toplevel(root)
    newWindowActor.configure(bg='black')
    newWindowActor.title("Actor Recommender")
    newWindowActor.geometry("700x700")
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    
    actor_tf = Entry(newWindowActor)
    actor_tf.var = tk.StringVar()
    actor_tf['textvariable'] = actor_tf.var
    actor_tf.var.trace_add('write', toggle_state)
    
    space = Label(newWindowActor, text="", bg = "black").pack()
    space = Label(newWindowActor, text="", bg = "black").pack()
    
    actor_lbl = Label(
    newWindowActor,
    text='Enter Actor',
    bg='#132530',
    fg='white',
    font = button_font)

    actor_lbl.pack()
    space = Label(newWindowActor, text="", bg = "black").pack()
    actor_tf.pack()
    space = Label(newWindowActor, text="", bg = "black").pack()
    
    btn = Button(
    newWindowActor,
    text='Suggestions by actor',
    relief=SOLID, state = 'disabled', activebackground='#bc272c',activeforeground='white', bg='#132530', fg='White',
    command=actor_show, font=button_font)
    btn.pack(pady=10)
    
    space = Label(newWindowActor, text="", bg = "black").pack()
    
    disp_tf = Text(newWindowActor)
    disp_tf.config(font = button_font, bg = "#132530", fg = "white")
    disp_tf.pack(pady=5)

In [ ]:
def openNewWindowGenre():
    
    def genre_recommender(genre_input):
        disp_tf.delete('1.0', END)
        genres_df.genres = genres_df.genres.str.replace(' ','')
        genre_list = genres_df[genres_df["genres"] == genre_input].values.tolist()
        num = 0
        for x,y,z,t in genre_list:
            if(num <20): 
                disp_tf.insert(END, y.title() + " | Critic's Score: " + str(t) + "%" + '\n\n')
                num = num +1
            else:
                break
                
    def genre_show():
        if (clicked.get() == "Choose Genre"):
            disp_tf.insert(END, "There is no entry!")
        else:
            input_ = clicked.get().lower()
            genre_recommender(input_)
            
    newWindowGenre = Toplevel(root)
    newWindowGenre.configure(bg='black')
    newWindowGenre.title("Genre Recommender")
    newWindowGenre.geometry("700x700")
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    
    #Dropdown menu options
    options = [
       'action&adventure', 'comedy', 'drama', 'sciencefiction&fantasy',
       'romance', 'classics', 'kids&family', 'mystery&suspense',
       'western', 'arthouse&international', 'horror',
       'faith&spirituality', 'animation', 'documentary',
       'specialinterest', 'musical&performingarts', 'sports&fitness',
       'television', 'cultmovies', 'anime&manga', 'gay&lesbian'
    ]

    clicked = StringVar(newWindowGenre)
    
    space = Label(newWindowGenre, text="", bg = "black").pack()
    space = Label(newWindowGenre, text="", bg = "black").pack()

    clicked.set("Choose Genre")

    drop = OptionMenu(newWindowGenre, clicked, *options)
    drop.config(bg='#132530', activebackground='#132530',activeforeground='white')
    drop.config(fg = "White")
    drop.config(font=button_font)
    
    menu = newWindowGenre.nametowidget(drop.menuname)
    menu.config(font=button_font)
    menu.config(bg='#132530', fg = "White")
      
    space = Label(newWindowGenre, text="", bg = "black").pack()
    drop.pack()
    space = Label(newWindowGenre, text="", bg = "black").pack()
   
    button = Button(newWindowGenre , text = "Suggestions by genre" , activebackground='#bc272c',activeforeground='white', bg='#132530', fg='White', command = genre_show, font=button_font).pack()
    
    space = Label(newWindowGenre, text="", bg = "black").pack()
    space = Label(newWindowGenre, text="", bg = "black").pack()
    
    disp_tf = Text(newWindowGenre)
    disp_tf.config(font = button_font, bg = "#132530", fg = "white")
    disp_tf.pack(pady=5)


In [ ]:
def openNewWindowDirector():
    
    def directors_recommender(director_input):
        disp_tf.delete('1.0', END)
        
        if(director_input in directors_df["directors"].values):
            director_list = directors_df[directors_df["directors"] == director_input].values.tolist()
            num = 0
            for x,y,z,t in director_list:
                if(num <20): 
                    disp_tf.insert(END, y.title() + " | Critic's Score: " + str(t) + "%"  + '\n\n')
                    num = num +1
                else:
                    break
        else:
            disp_tf.insert(END, "Sorry, there are no results for your entry!")
            
    def toggle_state(*_):
        if director_tf.var.get():
            btn['state'] = 'normal'
        else:
            btn['state'] = 'disabled'
    
    def clear_director():
        director_tf.delete(0, END)
    
    def director_show():
        if (director_tf.get() == ""):
            disp_tf.insert(END, "There is no entry!")
        else:
            input_ = director_tf.get().lower()
            directors_recommender(input_)
            clear_director()
            
    newWindowDirector = Toplevel(root)
    newWindowDirector.configure(bg='black')
    newWindowDirector.title("Director Recommender")
    newWindowDirector.geometry("700x700")
        
    director_tf = Entry(newWindowDirector)
    director_tf.var = tk.StringVar()
    director_tf['textvariable'] = director_tf.var
    director_tf.var.trace_add('write', toggle_state)
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    
    space = Label(newWindowDirector, text="", bg = "black").pack()
    space = Label(newWindowDirector, text="", bg = "black").pack()
    
    director_lbl = Label(
    newWindowDirector,
    text='Enter Director',
    bg='#132530',
    fg='white', 
    font = button_font)

    director_lbl.pack()
    space = Label(newWindowDirector, text="", bg = "black").pack()
    director_tf.pack()
    space = Label(newWindowDirector, text="", bg = "black").pack()
    
    btn = Button(
    newWindowDirector,
    text='Suggestions by director',
    relief=SOLID, state = 'disabled', activebackground='#bc272c',activeforeground='white', bg='#132530', fg='White',
    command=director_show, font=button_font)
    btn.pack(pady=10)
    
    space = Label(newWindowDirector, text="", bg = "black").pack()
    
    disp_tf = Text(newWindowDirector)
    disp_tf.config(font = button_font, bg = "#132530", fg = "white")
    disp_tf.pack(pady=5)


In [ ]:
def openNewWindowAddReview():
    wb = load_workbook('Audience_Review.xlsx')

    sheet = wb.active

    def excel():
        
        sheet.column_dimensions['A'].width = 30
        sheet.column_dimensions['B'].width = 50
        sheet.column_dimensions['B'].width = 20

        sheet.cell(row=1, column=1).value = "Movie Title"
        sheet.cell(row=1, column=2).value = "Review"
        sheet.cell(row=1, column=3).value = "Sentiment"

    def insert(movie, review, pred):

        current_row = sheet.max_row
        current_column = sheet.max_column

        sheet.cell(row=current_row + 1, column=1).value = movie
        sheet.cell(row=current_row + 1, column=2).value = review
        sheet.cell(row=current_row + 1, column=3).value = pred

        # save the file
        wb.save('Audience_Review.xlsx')

    
    def reviews_sentiment(movie_input, review_input):
        disp_tf.delete('1.0', END)
        
        if(movie_input in movies_data.index.unique()):
            test_matrix = vectorizer.transform([review_input])
            excel()
            predict = lr.predict(test_matrix)
            predict = re.sub(r"[\[\]]", " ", str(predict)).replace("'", "")
            insert(movie_input, review_input, predict)
            disp_tf.insert(END, 'Thank you for your' + str(predict).replace("'", "") + "review!")
        else:
            disp_tf.insert(END, "Sorry, the movie you entered doesn't exist!")
            
    def toggle_state(*_):
        if (movie_tf.var.get() and review_tf.get("1.0",'end-1c')):
            btn['state'] = 'normal'
        else:
            btn['state'] = 'disabled'
    
    def clear_reviews():
        movie_tf.delete(0, END)
        review_tf.config(state = NORMAL)
        review_tf.delete('1.0', END)
    
    def review_show():
        if (movie_tf.get() == "" or review_tf.get("1.0",'end-1c') == ""):
            disp_tf.insert(END, "There is no entry for either movie or review!")
        else:
            movie_name = movie_tf.get().lower()
            review = review_tf.get("1.0",'end-1c').lower()
            reviews_sentiment(movie_name, review)
            clear_reviews()
            
    newWindowAddReview = Toplevel(root)
    newWindowAddReview.configure(bg='black')
    newWindowAddReview.title("Add review")
    newWindowAddReview.geometry("700x700")
    
    movie_tf = Entry(newWindowAddReview)
    movie_tf.var = tk.StringVar()
    movie_tf['textvariable'] = movie_tf.var
    movie_tf.var.trace_add('write', toggle_state)
    button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")
    
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    
    movie_lbl = Label(
    newWindowAddReview,
    text='Type Movie Name',
    bg='black',
    fg='white', 
    font = button_font)

    movie_lbl.pack()
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    movie_tf.pack()
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    
    review_lbl = Label(
    newWindowAddReview,
    text='Type Your Review',
    bg='black',
    fg='white', 
    font = button_font)
    
    review_lbl.pack()
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    review_tf = Text(newWindowAddReview, height = 10, width = 100, bg = "#132530", fg = "white", font = button_font)
    review_tf.pack()
    review_tf.bind('<KeyRelease>', toggle_state)
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    
    btn = Button(
    newWindowAddReview,
    text='Enter to add your Review',
    relief=SOLID, state = 'disabled', activebackground='#bc272c',activeforeground='white', bg='#132530', fg='White',
    command=review_show, font=button_font)
    btn.pack(pady=10)
    
    space = Label(newWindowAddReview, text="", bg = "black").pack()
    
    disp_tf = Text(newWindowAddReview, height = 1, width = 100)
    disp_tf.config(font = button_font, bg = "black", fg = "white")
    disp_tf.pack()


In [ ]:
from openpyxl import *
from tkinter import *
import tkinter as tk
import tkinter.font as tkfont
      
root = Tk()
root.geometry("950x530")
root.title("Recommendation System for Movies")
    
bg = PhotoImage(file = "film.gif")

canvas1 = Canvas( root, width = 1000, height = 1000)

canvas1.pack(fill = "both", expand = True)

canvas1.create_image( 0, 0, image = bg, 
                 anchor = "nw")

bold_font = tkfont.Font(family="Helvetica", size=20, weight="bold")
canvas1.create_text(50, 70, text = "Select type of recommender...", anchor='w',  font=bold_font, fill="white")



button_font = tkfont.Font(family="Helvetica", size=10, weight="bold")

btn_genre = Button(root , text = "Select genre" , relief=SOLID, activebackground='#bc272c',activeforeground='white', bg='Black', fg='White', 
                   command = openNewWindowGenre, font=button_font)

btn_movie = Button(
root,
text='Suggestions by title',
relief=SOLID, activebackground='#bc272c',activeforeground='white', bg='Black', fg='White',
command=openNewWindowMovie, font=button_font)

btn_actor = Button(
root,
text='Suggestions by actor',
relief=SOLID, activebackground='#bc272c',activeforeground='white', bg='Black', fg='White',
command=openNewWindowActor, font=button_font)

btn_director = Button(
root,
text='Suggestions by director',
relief=SOLID, activebackground='#bc272c',activeforeground='white', bg='Black', fg='White',
command=openNewWindowDirector, font=button_font)

btn_add_review = Button(
root,
text='Add a review for our movies',
relief=SOLID, activebackground='#bc272c',activeforeground='white', bg='Black', fg='White',
command=openNewWindowAddReview, font=button_font)

  
button1_canvas = canvas1.create_window( 297, 230,
                                       anchor = "nw",
                                       window = btn_movie)

button2_canvas = canvas1.create_window( 178, 215, 
                                       anchor = "nw",
                                       window = btn_genre)

  
button3_canvas = canvas1.create_window( 485, 245, anchor = "nw",
                                       window = btn_actor)

button4_canvas = canvas1.create_window( 710, 255, anchor = "nw",
                                       window = btn_director)


button5_canvas = canvas1.create_window( 10, 460, anchor = "nw",
                                       window = btn_add_review)
root.mainloop()